### Loading documents

In [5]:
from langchain_community.document_loaders import PyPDFLoader
import os
import glob

# Ruta a la carpeta que contiene los PDFs
pdf_directory = "/home/cristian/projects/rag_pae/data/pdfs/amazonica"

def load_pdfs_from_directory(directory_path):
    """Carga todos los PDFs de un directorio con manejo de errores"""
    all_docs = []
    
    if not os.path.exists(directory_path):
        print(f"Error: La carpeta {directory_path} no existe")
        return all_docs
    
    # Obtener lista de archivos PDF
    pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))
    
    if not pdf_files:
        print(f"No se encontraron archivos PDF en {directory_path}")
        return all_docs
    
    print(f"Encontrados {len(pdf_files)} archivos PDF")
    
    # Cargar cada PDF individualmente
    for pdf_file in pdf_files:
        try:
            print(f"Cargando: {os.path.basename(pdf_file)}")
            loader = PyPDFLoader(pdf_file)
            docs = loader.load()
            all_docs.extend(docs)
            print(f"  ✓ Cargado: {len(docs)} páginas")
        except Exception as e:
            print(f"  ✗ Error cargando {os.path.basename(pdf_file)}: {e}")
    
    return all_docs

# Cargar todos los documentos
docs = load_pdfs_from_directory(pdf_directory)

print(f"\n Resumen:")
print(f"Total de páginas cargadas: {len(docs)}")

# Mostrar información de algunos documentos
if docs:
    print(f"\n📄 Primeros documentos:")
    for i, doc in enumerate(docs[:5]):
        source = os.path.basename(doc.metadata.get('source', 'Sin fuente'))
        page = doc.metadata.get('page', 'N/A')
        print(f"  {i+1}. {source} (página {page}) - {len(doc.page_content)} caracteres")

Encontrados 453 archivos PDF
Cargando: CL22.pdf
  ✓ Cargado: 14 páginas
Cargando: IE157.pdf
  ✓ Cargado: 7 páginas
Cargando: AN14.pdf
  ✓ Cargado: 13 páginas
Cargando: L1.pdf


KeyboardInterrupt: 

In [6]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")

In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding= embeddings, persist_directory="chroma_db")
retriever = vectorstore.as_retriever()

NameError: name 'docs' is not defined

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": """

 ¿De qué trata el foro con el título "Notas para una aproximación a la dinámica del conflicto armado en El Catatumbo?

"""})

results['answer']

### Using RAG Q&A

In [32]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Cargar la base de datos vectorial existente
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(persist_directory="/home/cristian/projects/rag_pae/chroma_db", embedding_function=embeddings)
retriever = vectorstore.as_retriever()

# Configurar el LLM
llm = OllamaLLM(model="llama3.2")

# Configurar el prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "Use line breaks to separte the sentences."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Crear las cadenas RAG
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Usar el RAG - ejemplo
results = rag_chain.invoke({
    "input": "dame un resumen Una isla a la deriva: creolización de la violencia en la isla de San Andrés"
})

print(results['answer'])

La isla de San Andrés ha experimentado una creolización de la violencia debido a su contexto único, donde los habitantes se sienten constantemente amenazados por la violencia y el narcotráfico. Esto ha llevado a la juventud a generar propuestas para combatir la violencia a través de la cultura. 

La isla está rodeada de noticias violentas y sus habitantes sobredimensionan los efectos, lo que ha llevado a una creolización de la violencia como parte de su identidad.
